In [1]:
import torch
import xml.etree.ElementTree as ET
from sentence_transformers import SentenceTransformer, util

In [2]:
ca_ru = ET.parse('data/full data/t1989-9-canada-russian-federation-bit-1989.xml')
ca_cz = ET.parse('data/full data/t1990-14-canada-czech-republic-bit-1990.xml')


In [9]:
root_ru = ca_ru.getroot()
root_cz = ca_cz.getroot()
article1_ru = root_ru[1][2][0]
article1_cz = root_cz[1][2][0]

sentences_ru = []
sentences_cz = []

for child in article1_ru:
   sentences_ru.append(child.text)
for child in article1_cz:
   sentences_cz.append(child.text)

print(len(sentences_cz))


16


In [10]:
sentences_ru

['For the purpose of this Agreement:',
 '(a) The term "territory" means the territory of Canada or the territory of the Union of Soviet Socialist Republics respectively, as well as those maritime areas, including the seabed and subsoil adjacent to the outer limit of the territorial sea of either of the above territories, over which the State concerned exercises, in accordance with international law, sovereign rights for the purpose of exploration and exploitation of the natural resources of such areas;',
 '(b) The term "investment" means any kind of asset invested either directly, or indirectly through an investor of a third State, by an investor of one Contracting Party in the territory of the other Contracting Party and in particular, though not exclusively, shall include:',
 '(i) Any movable and immovable property and any related property rights, such as mortgages;',
 '(ii) Shares, stock, bonds and debentures or any other form of participation in a company, business enterprise or jo

In [4]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

sentence_ru_embeddings = model.encode(sentences_ru)
sentence_cz_embeddings = model.encode(sentences_cz)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [5]:
for id, embd in enumerate(sentence_cz_embeddings):

    scores = util.cos_sim(embd, sentence_ru_embeddings)

    if max(scores[0])<0.8 :
        print(sentences_cz[id])


(d) The term "territory" means:
(ii) In respect of the Czech and Slovak Federal Republic, the territory of the Czech and Slovak Federal Republic.
